In [1]:
import keras
import numpy as np
import os
import corner
import matplotlib.pyplot as plt

Using TensorFlow backend.
/media/yqin/81614149-2ed8-4c76-82e1-c46763d086fa/pregi/anaconda3/envs/21CNN_GPU_NEW/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/media/yqin/81614149-2ed8-4c76-82e1-c46763d086fa/pregi/anaconda3/envs/21CNN_GPU_NEW/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/media/yqin/81614149-2ed8-4c76-82e1-c46763d086fa/pregi/anaconda3/envs/21CNN_GPU_NEW/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a f

In [10]:
#setup of GPU

############################
### KERAS BACKEND and LIMIT GPU
############################
os.environ['KERAS_BACKEND'] = 'tensorflow'
import tensorflow as tf

#setting up GPU
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 1. #setting the percentage of GPU usage
config.gpu_options.visible_device_list = "0" #for picking only some devices
config.gpu_options.allow_growth = True

#passing tf session to keras!
from keras.backend.tensorflow_backend import set_session
set_session(tf.Session(config=config)) 

In [11]:
#loading model

model_loc = "CNN/data_save/NGillet/tophat22_Z30_meanZ_deeper_good/"
model_prename = "tophat22_Z30_meanZ_deeper2D_Filter55_1batchNorm"

# load json and create model
with open(f"{model_loc}{model_prename}.json", 'r') as f:
    model = keras.models.model_from_json(f.read())
# load weights into new model
model.load_weights(f"{model_loc}{model_prename}_weights.h5")
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
PreConv-1 (Conv2D)           (None, 46, 1049, 16)      416       
_________________________________________________________________
PreConv-2 (Conv2D)           (None, 42, 1045, 32)      12832     
_________________________________________________________________
PrePool-2 (MaxPooling2D)     (None, 21, 522, 32)       0         
_________________________________________________________________
Conv-1 (Conv2D)              (None, 17, 518, 32)       25632     
_________________________________________________________________
Conv-2 (Conv2D)              (None, 13, 514, 64)       51264     
_________________________________________________________________
Pool-2 (MaxPooling2D)        (None, 6, 257, 64)        0         
_________________________________________________________________
Conv-3 (Conv2D)              (None, 1, 252, 64)        147520    
__________

In [12]:
DataFilepath = "../data/"
DataXname = "database5_float32.npy"
RemovedMean = True
if RemovedMean:
    rm = "meanZ_"
else:
    rm = ""
pTrain = 0.8
pDev = 0.1
pTest = 0.1
tophat = [2, 2]
Zmax = 30
DataType = f"tophat{tophat[0]}{tophat[1]}_Z{Zmax}_{rm}" + DataXname
print(DataType)
TDTfile = "{}/{}_{:.1f}_" + DataType
# trainX = np.load(DataFilepath + TDTfile.format("train", "X", pTrain))
# trainY = np.load(DataFilepath + TDTfile.format("train", "Y", pTrain))
testX  = np.load(DataFilepath + TDTfile.format("test", "X", pTest))
testY  = np.load(DataFilepath + TDTfile.format("test", "Y", pTest))
# devX   = np.load(DataFilepath + TDTfile.format("dev", "X", pDev))
# devY   = np.load(DataFilepath + TDTfile.format("dev", "Y", pDev))

tophat22_Z30_meanZ_database5_float32.npy


In [19]:
testY.shape

(20000, 4)

In [20]:
X = testX[..., np.newaxis]
Y = testY[..., np.newaxis]

In [21]:
predY = model.predict(X)

In [35]:
#fiducial parameters ["ZETA","TVIR_MIN","L_X","NU_X_THRESH"] Brad used in MCMC
FaintFiducialY = [ 30  , 4.70, 40, 500 ]
BrightFiducialY = [ 200 , 5.48, 40, 500 ]

#E0 gledam na 500 vjerojatno, 0.5 keV

In [36]:
Parameters = np.load("../data/databaseParams_float32.npy")

In [48]:
for i in range(10000):
    FaintDif = FaintFiducialY - Parameters[i]
    BrightDif = BrightFiducialY - Parameters[i]
    if np.sqrt(FaintDif @ FaintDif) < 5:
        print(f"Faint Fiducial Index = {i}, {np.sqrt(FaintDif @ FaintDif)}")
    if np.sqrt(BrightDif @ BrightDif) < 5:
        print(f"Bright Fiducial Index = {i}, {np.sqrt(BrightDif @ BrightDif)}")

Bright Fiducial Index = 1770, 3.740047199493469
Faint Fiducial Index = 1789, 3.402656478041788
Bright Fiducial Index = 3829, 3.762859535785915
Bright Fiducial Index = 5932, 4.134607599013513


In [50]:
print(FaintFiducialY, Parameters[1789])
print(BrightFiducialY, Parameters[1770])
print(BrightFiducialY, Parameters[3829])


[30, 4.7, 40, 500] [ 30.943579   5.372314  38.932    496.9842  ]
[200, 5.48, 40, 500] [196.35991    4.968501  40.01446  499.31018 ]
[200, 5.48, 40, 500] [202.77774    4.343854  39.76903  502.25812 ]


In [ ]:
if 

In [31]:
import json
with open("../data/databaseParams_min_max.txt", "r") as f:
    ParametersSummary = json.load(f)
    print(json.dumps(ParametersSummary, indent=4))

{
    "parameters": [
        "ZETA",
        "TVIR_MIN",
        "L_X",
        "NU_X_THRESH"
    ],
    "min": [
        10.023063659667969,
        4.000161170959473,
        38.00041961669922,
        100.13026428222656
    ],
    "max": [
        249.9544677734375,
        5.998933792114258,
        41.99989700317383,
        1499.37109375
    ]
}
